<a href="https://colab.research.google.com/github/angelinawong1210/VietnameseFakeNews/blob/main/FakeNews_SVM_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
truenews = pd.read_csv('/content/Fake.csv')
fakenews = pd.read_csv('/content/Real.csv')

In [ ]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [ ]:
# Combine the 2 DataFrames into a single data frame
news = pd.concat([truenews, fakenews])
news["Article"] = news["tittle"] + news["text"]
news.sample(frac = 1) #Shuffle 100%

,tittle,text,domain,topic,True/Fake,Article
29,"Những món ăn ""thần kỳ"" giúp sĩ tử ""thi đâu đậu...",Ăn đậu để thi đâu đậu đó hay ăn bắp để bài là...,thoibao.today,Đời sống,True,"Những món ăn ""thần kỳ"" giúp sĩ tử ""thi đâu đậu..."
128,VKS đề nghị bác đơn kêu oan của ông Lê Tùng Vân,"VKS cho rằng, tòa cấp sơ thẩm đã xét xử ông Lê...",vnexpress.net,An ninh,Fake,VKS đề nghị bác đơn kêu oan của ông Lê Tùng Vâ...
9,Bức vẽ giúp bạn đánh giá mức độ stress của bản...,"Nhìn bức tranh, bạn có thể thấy vòng tròn đan...",suckhoe.vnexpress.net,Y tế,True,Bức vẽ giúp bạn đánh giá mức độ stress của bản...
13,Đề xuất cấm tất cả công chức Hà Nội đổ xăng tạ...,"Theo một nguồn tin, chiều ngày 11/10 UBND TP....",phapluat.news,Chính trị,True,Đề xuất cấm tất cả công chức Hà Nội đổ xăng tạ...
95,Chuyện li kì người phụ nữ cứu rắn hổ chúa và b...,Rắn hổ mang (danh pháp khoa học: Naja atra) l...,tinvn.info,Đời sống,True,Chuyện li kì người phụ nữ cứu rắn hổ chúa và b...
...,...,...,...,...,...,...
89,Ôtô đi ngược chiều bị phạt 800.000 đồng,Theo khoản 1 Điều 9 Luật Giao thông đường bộ ...,vnexpress.net,Giao thông,Fake,Ôtô đi ngược chiều bị phạt 800.000 đồng Theo k...
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,tintucqpvn.net,Giải trí,True,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!Ch...
51,Xã hội đen Đài Loan mở tiệm mỳ miễn phí cho ng...,\n Xã hội đen Italy dụ người nghèo hủy hoại ch...,vnexpress.net,Đời sống,Fake,Xã hội đen Đài Loan mở tiệm mỳ miễn phí cho ng...
140,"Đòi quan hệ nhiều lần mà không có quà, nữ sinh...","Sau nhiều lần hứa tặng quà, cho tiền ân ái như...",NaN,Đời sống,True,"Đòi quan hệ nhiều lần mà không có quà, nữ sinh..."


In [ ]:
import nltk 
nltk.download('stopwords')
#Data Cleaning
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def process_text(s):

    # Check string to see if they are a punctuation
    nopunc = [char for char in s if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in nopunc.split()]
    return clean_string

In [ ]:
# Tokenize the Article
#rerun, takes LOOOONG
news['Clean Text'] = news['Article'].apply(process_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_)) #Total vocab words


298


In [ ]:
#Bag-of-Words (bow) transform the entire DataFrame of text
news_bow = bow_transformer.transform(news['Clean Text'])

In [ ]:
print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

Shape of Sparse Matrix:  (299, 298)
Amount of Non-Zero occurences:  299


In [ ]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(299, 298)


In [ ]:
#Train SVM model
from sklearn.svm import SVC
fakenews_detect_model = SVC(kernel = 'linear', random_state = 0).fit(news_tfidf, news['True/Fake'])


In [ ]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))


              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       150
        True       1.00      1.00      1.00       149

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299



In [ ]:
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

209 90 299


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', SVC(kernel = 'linear', random_state = 0)),  
])
pipeline.fit(news_train,text_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x7f7d09944280>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', SVC(kernel='linear', random_state=0))])

In [ ]:
prediction = pipeline.predict(news_test)

In [ ]:
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.92      0.80      0.85        44
        True       0.83      0.93      0.88        46

    accuracy                           0.87        90
   macro avg       0.87      0.87      0.87        90
weighted avg       0.87      0.87      0.87        90



# Mục mới